### Install & import dependencies

In [1]:
%pip install ndlib --quiet
%pip install matplotlib --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import networkx as nx
from custommodels.SpreadingModels import CascadeModel, ThresholdModel
from custommodels.Pickle import pickle, load
import random
import matplotlib.pyplot as plt
import numpy as np

### Load data

In [3]:
path = "../data/loc-brightkite_edges.txt.gz"
G = nx.read_edgelist(path, create_using=nx.Graph(), nodetype=int)

# Step 2. Make the disease spread on all four models

1. Adding the the code for the models (threshold and cascade) and the code to import the pickle with degree and betweeness


### Highest degree nodes:

In [4]:
sorted_degree_list = load("sorted_degree")

Found data, loading from picklejar


In [13]:
def get_top_degree_nodes(percentage):
    top_degree_nodes = []
    n_selected = int(len(sorted_degree_list)*percentage)
    for b in sorted_degree_list[:n_selected]:

        top_degree_nodes.append(str(b[0]))
    return top_degree_nodes

### Highest betweeness centrality nodes:

In [6]:
betweenness = (load("betweenness_centrality"))
betweenness = list(betweenness.items())
sorted_betweenness = sorted(betweenness, key=lambda x: x[1], reverse=True)


Found data, loading from picklejar


In [8]:
def get_top_betweenness_nodes(percentage):
    top_betweenness_nodes = []
    n_selected = int(len(sorted_betweenness)*percentage)
    for b in sorted_betweenness[:n_selected]:

        top_betweenness_nodes.append(str(b[0]))
    return top_betweenness_nodes

['40', '191', '159', '108', '44', '36', '116', '35', '49', '405', '1100', '2625', '833', '212', '210', '102', '195', '2349', '6991', '446', '163', '3', '156', '634', '207', '176', '1076', '174', '1878', '1302', '6365', '651', '719', '30', '2189', '2934', '298', '901', '1093', '214', '7', '3245', '2966', '2868', '1936', '161', '157', '6589', '948', '112', '1877', '27', '168', '1143', '646', '680', '1455', '1501', '2314', '148', '6640', '229', '208', '1168', '1386', '740', '2180', '1057', '1895', '146', '2915', '686', '1914', '1955', '69', '2168', '1898', '8440', '661', '3596', '1304', '1138', '4841', '3801', '2261', '3109', '295', '652', '2278', '4629', '424', '1910', '1158', '730', '189', '620', '2147', '2836', '5215', '754', '1786', '3309', '2284', '375', '1082', '45', '164', '7084', '527', '1864', '83', '14814', '2874', '2360', '1901', '3305', '915', '3558', '235', '3154', '417', '209', '182', '2809', '970', '2971', '143', '1771', '1318', '241', '1531', '29', '647', '639', '1542', '6

In [9]:
def get_random_nodes(percentage):
    random_nodes = []
    n_selected = int(len(G.nodes())*percentage)
    for b in random.sample(G.nodes(), n_selected):
        random_nodes.append(str(b))
    return random_nodes

/tmp/ipykernel_15712/1594282658.py:5: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  for b in random.sample(G.nodes(), n_selected):


In [ ]:
def plot_resulst(results, num_initial_infected):
    final_sus = list(results[-1][0].values()).count(0)
    final_inf = list(results[-1][0].values()).count(1)
    final_rec = list(results[-1][0].values()).count(2)
    print(f"Final susceptible: {final_sus}")
    print(f"Final infected: {final_inf}")
    print(f"Final recovered: {final_rec}")
    # Plotting  the graph
    num_susceptible = [len(G.nodes) - num_initial_infected]
    num_infected = [num_initial_infected]
    num_recovered = [0]
    for state, i, r in results:
        num_susceptible.append(list(state.values()).count(0))
        num_infected.append(list(state.values()).count(1))
        num_recovered.append(list(state.values()).count(2))

    num_susceptible = np.array(num_susceptible)
    num_infected = np.array(num_infected)
    num_recovered = np.array(num_recovered)

    # get proportion
    prop_susceptible = num_susceptible / len(G.nodes)
    prop_infected = num_infected / len(G.nodes)
    prop_recovered = num_recovered / len(G.nodes)

    plt.plot(prop_susceptible, label="Susceptible", color="blue")
    plt.plot(prop_infected, label="Infected", color="red")
    plt.plot(prop_recovered, label="Recovered", color="green")
    plt.legend()
    plt.xlabel("Time")
    plt.ylabel("Number of nodes")
    plt.show()

## Cascade model infecting top X% highest degree nodes 

In [ ]:
model = CascadeModel(G)
beta, gamma = 0.20, 1
model.set_initial_infected(top_degree_nodes)
model.set_parameters(beta, gamma)
results = model.iterate(100)
# Print end result
final_sus = list(results[-1][0].values()).count(0)
final_inf = list(results[-1][0].values()).count(1)
final_rec = list(results[-1][0].values()).count(2)
print(f"Final susceptible: {final_sus}")
print(f"Final infected: {final_inf}")
print(f"Final recovered: {final_rec}")
model.plot_sir_graph()

## Threshold model infecting top 10% highest degree nodes 

## Cascade model with top 10% highest betweeness centrality nodes

In [ ]:
cascade_degree = CascadeModel(G)
beta, gamma = 0.21204, 0.1
cascade_degree.set_parameters(beta, gamma)
# Get random 1% of nodes as initial infected
percentage = 0.05
initial_infected = top_betweenness_nodes
cascade_degree.set_initial_infected(initial_infected)
results = cascade_degree.iterate(40)

# Info
# Number at final step
final_sus = list(results[-1][0].values()).count(0)
final_inf = list(results[-1][0].values()).count(1)
final_rec = list(results[-1][0].values()).count(2)
print(f"Final susceptible: {final_sus}")
print(f"Final infected: {final_inf}")
print(f"Final recovered: {final_rec}")

In [ ]:
# Plotting  the graph
num_susceptible = [len(G.nodes) - len(initial_infected)]
num_infected = [len(initial_infected)]
num_recovered = [0]
for state, i, r in results:
    num_susceptible.append(list(state.values()).count(0))
    num_infected.append(list(state.values()).count(1))
    num_recovered.append(list(state.values()).count(2))

num_susceptible = np.array(num_susceptible)
num_infected = np.array(num_infected)
num_recovered = np.array(num_recovered)
# print(num_susceptible, num_infected, num_recovered)

# get proportion
prop_susceptible = num_susceptible / len(G.nodes)
prop_infected = num_infected / len(G.nodes)
prop_recovered = num_recovered / len(G.nodes)

plt.plot(prop_susceptible, label="Susceptible", color="blue")
plt.plot(prop_infected, label="Infected", color="red")
plt.plot(prop_recovered, label="Recovered", color="green")
plt.legend()
plt.xlabel("Time")
plt.ylabel("Number of nodes")
plt.show()

## Threshold model with top 10% highest betweeness centrality nodes

In [ ]:
model = ThresholdModel(G)
theta, gamma = 0.21204, 0.1
model.set_parameters(theta, gamma)
# Initial infected
percentage = 0.01
initial_infected = top_betweenness_nodes
model.set_initial_infected(initial_infected)
results = model.iterate(40)

# Info
# Number at final step
final_sus = list(results[-1][0].values()).count(0)
final_inf = list(results[-1][0].values()).count(1)
final_rec = list(results[-1][0].values()).count(2)
print(f"Final susceptible: {final_sus}")
print(f"Final infected: {final_inf}")
print(f"Final recovered: {final_rec}")

In [ ]:
# Plotting  the graph
num_susceptible = [len(G.nodes) - len(initial_infected)]
num_infected = [len(initial_infected)]
num_recovered = [0]
for state, i, r in results:
    num_susceptible.append(list(state.values()).count(0))
    num_infected.append(list(state.values()).count(1))
    num_recovered.append(list(state.values()).count(2))

num_susceptible = np.array(num_susceptible)
num_infected = np.array(num_infected)
num_recovered = np.array(num_recovered)
# print(num_susceptible, num_infected, num_recovered)

# get proportion
prop_susceptible = num_susceptible / len(G.nodes)
prop_infected = num_infected / len(G.nodes)
prop_recovered = num_recovered / len(G.nodes)

plt.plot(prop_susceptible, label="Susceptible", color="blue")
plt.plot(prop_infected, label="Infected", color="red")
plt.plot(prop_recovered, label="Recovered", color="green")
plt.legend()
plt.xlabel("Time")
plt.ylabel("Number of nodes")
plt.show()